In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

from sklearn.datasets import load_digits
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

from sklearn.ensemble import RandomForestClassifier

np.random.seed(42)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
df = pd.read_csv('module7_HR-dataset.csv')

target = 'left'
features = df.columns.drop(target)
features = features.drop('empid')  # Удалим идентификатор пользователя как нерепрезентативный признак
print(features)

X, y = df[features].copy(), df[target]

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'dept', 'salary'],
      dtype='object')


In [3]:
df

,empid,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,dept,salary
0,1001,0.38,0.53,2,157,3,0,1,0,sales,low
1,1002,0.80,0.86,5,262,6,0,1,0,sales,medium
2,1003,0.11,0.88,7,272,4,0,1,0,sales,medium
3,1004,0.72,0.87,5,223,5,0,1,0,sales,low
4,1005,0.37,0.52,2,159,3,0,1,0,sales,low
...,...,...,...,...,...,...,...,...,...,...,...
14994,15995,0.40,0.57,2,151,3,0,1,0,support,low
14995,15996,0.37,0.48,2,160,3,0,1,0,support,low
14996,15997,0.37,0.53,2,143,3,0,1,0,support,low
14997,15998,0.11,0.96,6,280,4,0,1,0,support,low


In [4]:
X['dept'].value_counts().get

<bound method NDFrame.get of sales          4140
technical      2720
support        2229
IT             1227
product_mng     902
marketing       858
RandD           787
accounting      767
hr              739
management      630
Name: dept, dtype: int64>

In [5]:
salary_ordinals = {'low': 1, 'medium': 2, 'high': 3}

X['dept'] = X['dept'].apply(X['dept'].value_counts().get)
X['salary'] = X['salary'].apply(salary_ordinals.get)

In [6]:
X

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,0.38,0.53,2,157,3,0,0,4140,1
1,0.80,0.86,5,262,6,0,0,4140,2
2,0.11,0.88,7,272,4,0,0,4140,2
3,0.72,0.87,5,223,5,0,0,4140,1
4,0.37,0.52,2,159,3,0,0,4140,1
...,...,...,...,...,...,...,...,...,...
14994,0.40,0.57,2,151,3,0,0,2229,1
14995,0.37,0.48,2,160,3,0,0,2229,1
14996,0.37,0.53,2,143,3,0,0,2229,1
14997,0.11,0.96,6,280,4,0,0,2229,1


In [7]:
scaler = StandardScaler()
X = pd.DataFrame(data=scaler.fit_transform(X), columns=X.columns)

In [8]:
X

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,dept,salary
0,-0.936495,-1.087275,-1.462863,-0.882040,-0.341235,-0.411165,-0.147412,1.370267,-0.933368
1,0.752814,0.840707,0.971113,1.220423,1.713436,-0.411165,-0.147412,1.370267,0.636092
2,-2.022479,0.957554,2.593763,1.420657,0.343655,-0.411165,-0.147412,1.370267,0.636092
3,0.431041,0.899131,0.971113,0.439508,1.028546,-0.411165,-0.147412,1.370267,-0.933368
4,-0.976716,-1.145699,-1.462863,-0.841993,-0.341235,-0.411165,-0.147412,1.370267,-0.933368
...,...,...,...,...,...,...,...,...,...
14994,-0.856051,-0.853580,-1.462863,-1.002181,-0.341235,-0.411165,-0.147412,-0.064033,-0.933368
14995,-0.976716,-1.379394,-1.462863,-0.821970,-0.341235,-0.411165,-0.147412,-0.064033,-0.933368
14996,-0.976716,-1.087275,-1.462863,-1.162368,-0.341235,-0.411165,-0.147412,-0.064033,-0.933368
14997,-2.022479,1.424944,1.782438,1.580845,0.343655,-0.411165,-0.147412,-0.064033,-0.933368


In [9]:
def estimate_accuracy(clf, X, y, cv=5):
    return cross_val_score(clf, X, y, cv=5, scoring='accuracy').mean()

In [10]:
tree = DecisionTreeClassifier(max_depth=30)
print("Decision tree:", estimate_accuracy(tree, X, y))

Decision tree: 0.9731310659108592


In [11]:
bagging_trees = BaggingClassifier(tree)
print("Decision tree bagging:", estimate_accuracy(bagging_trees, X, y))

Decision tree bagging: 0.9880660886962321


In [12]:
random_tree = DecisionTreeClassifier(max_features=int(np.sqrt(len(features))), max_depth=30)
print("Random tree:", estimate_accuracy(random_tree, X, y))

Random tree: 0.9778657330221184


In [13]:
bagging_random_trees = BaggingClassifier(random_tree)
print("Random tree bagging:", estimate_accuracy(bagging_random_trees, X, y))

Random tree bagging: 0.9902662443036567


Стандартная эвристика: в задаче классификации брать квадратный корень числа признаков, а в задаче регрессии — треть числа признаков.

In [14]:
random_forest = RandomForestClassifier(
    n_estimators=100,
    n_jobs=-1,
    max_features=int(np.sqrt(len(features))),
    max_depth=30)
print("Random Forest:", estimate_accuracy(random_forest, X, y))

Random Forest: 0.9920663109925532


In [15]:
random_forest = RandomForestClassifier(
    n_estimators=100,
    max_features=int(np.sqrt(len(features))),
    max_depth=30,
    oob_score=True,
    n_jobs=-1
)
random_forest.fit(X, y)
random_forest.oob_score_.mean()

0.9929995333022201

In [16]:
lr = LogisticRegression(solver='saga', max_iter=200)
lr.fit(X, y)
print("LR:", estimate_accuracy(lr, X, y))

LR: 0.7709770367900411


In [17]:
random_logreg = BaggingClassifier(
    lr,
    n_estimators=10,
    n_jobs=-1,
    random_state=42
)
print("Bagging for LR:", estimate_accuracy(random_logreg, X, y))

Bagging for LR: 0.7701104368122708


In [18]:
random_logreg = BaggingClassifier(
    lr,
    n_estimators=10,
    n_jobs=-1,
    max_features=0.5,
    random_state=42
)
print("Bagging for LR:", estimate_accuracy(random_logreg, X, y))

Bagging for LR: 0.7569754140268978


In [19]:
def plot_predictions(X, y, clf, proba=False, points_size=7, xlabel='x', ylabel='y'):
    """Fits the classifier on the data (X, y) and plots the result on a 2-D plane."""
    def get_grid(data):
        x_std, y_std = data.std(axis=0)
        x_min, x_max = data[:, 0].min() - x_std / 2, data[:, 0].max() + x_std / 2
        y_min, y_max = data[:, 1].min() - y_std / 2, data[:, 1].max() + y_std / 2
        return np.meshgrid(np.linspace(x_min, x_max, num=200),
                           np.linspace(y_min, y_max, num=200))
    
    clf.fit(X, y)
    xx, yy = get_grid(X)
    if proba:
        predicted = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1].reshape(xx.shape)
    else:
        predicted = clf.predict(np.c_[xx.ravel(), yy.ravel()]).reshape(xx.shape)
        
    plt.figure(figsize=(10.0, 10.0))
    plt.pcolormesh(xx, yy, predicted, cmap=plt.cm.coolwarm, alpha=0.1)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=points_size, cmap=plt.cm.coolwarm, alpha=0.90)
    plt.ylim([yy.min(),yy.max()])
    plt.xlim([xx.min(),xx.max()])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    
    return clf

In [20]:
digits = load_digits()

In [21]:
X = digits.data
y = digits.target

In [22]:
DTC = DecisionTreeClassifier()
cvs = cross_val_score(DTC, X, y, cv=10)
cvs

array([0.78888889, 0.85      , 0.83888889, 0.78888889, 0.78333333,
       0.87777778, 0.87777778, 0.81564246, 0.80446927, 0.81005587])

In [23]:
cvs.mean()

0.8235723153320919

In [24]:
BC = BaggingClassifier(DTC, n_estimators=100)
cvs = cross_val_score(BC, X, y, cv=10)
cvs

array([0.86111111, 0.94444444, 0.91111111, 0.93333333, 0.91666667,
       0.98333333, 0.96666667, 0.91620112, 0.87150838, 0.91061453])

In [25]:
cvs.mean()

0.9214990689013035

In [26]:
BC = BaggingClassifier(DTC, n_estimators=100, max_features=int(np.sqrt(X.shape[1])))
cvs = cross_val_score(BC, X, y, cv=10)
cvs

array([0.91666667, 0.96666667, 0.93333333, 0.89444444, 0.94444444,
       0.95      , 0.96111111, 0.98882682, 0.91061453, 0.91061453])

In [27]:
cvs.mean()

0.9376722532588453

In [28]:
BC = BaggingClassifier(DecisionTreeClassifier(max_features='sqrt'), n_estimators=100)
cvs = cross_val_score(BC, X, y, cv=10)
cvs

array([0.9       , 0.97777778, 0.95      , 0.94444444, 0.96111111,
       0.97222222, 0.98333333, 0.96648045, 0.91620112, 0.93854749])

In [29]:
cvs.mean()

0.9510117939168218

In [46]:
RFC = RandomForestClassifier(n_estimators=50, max_depth=100)
cvs = cross_val_score(RFC, X, y, cv=10)
cvs.mean()

0.9482309124767225